前面的几种文本表示方法：
- One-hot
- Bag of Words
- N-gram
- TF-IDF

或多或少存在一定的**问题**：
- 转换得到的向量维度很高，需要较长的训练时间
- 没有考虑单词与单词之间的关系，只是进行了统计

与这些表示方法不同，**深度学习也可以用于文本表示**（与传统机器学习不同，深度学习既提供特征提取功能，也可以完成分类的功能），还可以将词向量**映射到一个低纬空间**。

其中比较典型的例子有：FastText、Word2Vec和Bert

### FastText
FastText是一种典型的深度学习**词向量的表示方法**（FastText是一个三层的神经网络，输入层、隐含层和输出层），通过Embedding层将单词映射到稠密空间（低纬度），然后将句子中所有的单词在Embedding空间中进行平均，进而完成分类操作。

FastText在文本分类任务上，是优于TF-IDF的：
- FastText用单词的Embedding叠加获得的文档向量，将相似的句子分为一类
- FastText学习到的Embedding空间维度比较低，可以快速进行训练

Fasttext安装报错:
error: Microsoft Visual C++ 14.0 is required. Get it with "Build Tools for Visual Studio": https://visualstudio.microsoft.com/downloads/

原因在于该包只支持mac和linux，不支持windows

解决办法：
https://blog.csdn.net/qq_40535327/article/details/104531649

https://blog.csdn.net/IOT_victor/article/details/90449989?utm_medium=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.edu_weight&depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromMachineLearnPai2-2.edu_weight

In [2]:
# fasttext库既包括文本表示，也包括文本分类
import fasttext 
import pandas as pd
from sklearn.metrics import f1_score

In [5]:
train_all = pd.read_csv('./train_set.csv.zip', sep='\t', nrows=15000)
# 划分训练集和验证集
train, valid = train_all.iloc[:-5000], train_all.iloc[-5000:]

前一万条数据训练，因为训练是监督学习，要用到 text 对应的 label，所以不加入验证集的数据来进行文本表示

In [6]:
%%time
# 将类别转换为FastText需要的格式: __label__
train['label_fasttext'] = '__label__' + train['label'].astype(str)

# 将Fasttext所需的数据形式（特征和类别）存到文件里
train[['text','label_fasttext']].to_csv('train_fasttext.csv', index=None, header=None, sep='\t')

# 模型的输入是文件而不是数据变量
model = fasttext.train_supervised('train_fasttext.csv', lr=1.0, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss="hs")

d:\softwares\anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Wall time: 41 s


后5000条数据验证

In [7]:
%%time

# 将预测的Fasttext格式的类别转换为正常的类别格式
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in valid['text']]
print(f1_score(valid['label'].values.astype(str), val_pred, average='macro'))

0.823134217943867
Wall time: 1.51 s


### 训练和验证
#### 如何使用验证集调参
在使用TF-IDF和FastText中，有一些模型的参数需要选择，这些参数会在一定程度上影响模型的精度，如何选择这些参数：
- 通过阅读文档，要弄清楚这些参数的大致含义，哪些参数会增加模型的复杂度
- 通过在验证集上进行验证模型精度，找到模型是过拟合，还是欠拟合

#### 十折交叉验证
把训练集数据分成十份，9份训练，一份验证。需要注意，每折的划分必须**保证标签的分布与整个数据集的分布一致**。

### 本章作业
- 阅读FastText的文档，尝试修改参数，得到更好的分数
- 基于验证集的结果调整超参数，使得模型性能更优